# Crawl data

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains

# other necessary ones
import re
from random import random
import csv
# import urllib.request
# import json
# import datetime
# import time


option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

browser = webdriver.Edge(executable_path="msedgedriver.exe", options=option)

c:\Users\hieuc\Documents\Projects\USC\DataFest2022\craw-venv\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [ ]:
# Extract links to issues on Github:
browser.get("https://github.com/SeleniumHQ/selenium/issues?page=416&q=")
links = []
while browser.find_element(By.CLASS_NAME, "next_page"):
    link_els = browser.find_elements(By.CSS_SELECTOR, ".d-block.d-md-none.position-absolute.top-0.bottom-0.left-0.right-0")
    for l in link_els:
        links.append(l.get_attribute("href"))
    next_page_url = browser.find_element(By.CLASS_NAME, "next_page")
    next_page_url = next_page_url.get_attribute("href")
    try:
        browser.get(next_page_url)
    except:
        print("Exception", next_page_url)
        break

# print(links)

In [19]:
ls = open("issues-urls.txt", "a")
for l in links:
    ls.write(l + "\n")

In [ ]:
# Extract conversations from issues and write to file dataset-selenium.txt

convs = []
dataset = open("dataset-selenium.txt", "a", encoding="utf-8")
count = 0
for wpage in links:
    count += 1
    if count < 2955:
        continue
    browser.get(wpage)
    conv = browser.find_elements(By.CLASS_NAME, "edit-comment-hide")
    c = []
    for turn in conv:
        c.append(turn.text.replace("\n", " "))
    # convs.append(c)
    # print("\n\n", "_eot_".join(c))
    dataset.write("_eot_".join(c) + "\n")
    # break

# print(convs)

# Clean data

In [ ]:
emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

convs = []
flag = False
dataset = open("dataset-selenium.txt", "r", encoding="utf-8")
count = 0
for conv in dataset:
    conv = conv.lower()
    # Remove unhelpful starting phrases
    conv = re.sub("What happened?".lower(), "", conv)
    conv = re.sub("🐛 Bug Report".lower(), "", conv)
    conv = re.sub("💬 Questions and Help".lower(), "", conv)
    conv = re.sub("^Decription".lower(), "", conv)
    conv = re.sub("🚀 Feature Proposal".lower(), "", conv)
    conv = re.sub("💥 Regression Report".lower(), "", conv)
    
    # Remove "<username> reacted with <emojiname> emoji"
    emojis = ["😄", "👍", "👎", "🎉", "😕", "❤️", "🚀", "👀"]
    for turn in conv.split("_eot_"):
        if len(turn) < 2:
            continue
        parts = turn.split()
        length = len(parts)
        if parts[-1] == "emoji":
            for i, el in enumerate(parts):
                if el in emojis and i < length-1:
                    # print(f"el is {el} is in emojis")
                    # print(f"[parts[i+5], parts[i+3]] is {[parts[i+5], parts[i+3]]}")
                    if not parts[i+1].isdigit():
                        continue
                    if parts[i+1] not in ["1", "2"]: # i+2, i+3, i+4 --> [i+2:i+1+num]
                        # print("in!")
                        # print(f"parts[i+1] is {parts[i+1]}")
                        count = 0
                        digit = int(parts[i+1])
                        digit = 10 if digit > 10 else digit 
                        for name in parts[i+2:i+2 + digit]:
                            # print(f"name is {name}")
                            if "," == name[-1]:
                                count += 1
                        digit = digit -1 if digit < 10 else digit
                        if count == digit:
                            start_index = i
                            break
                    elif "reacted" in [parts[i+5], parts[i+3]]:
                        # print("in 1 2!")
                        start_index = i
                        break
            parts = " ".join(parts[:start_index])
    # Remove emojis
    conv = re.sub(emoj, '', conv)
    conv = re.sub("====", "", conv)
    conv = re.sub("!!!!", "", conv)
    conv = re.sub("----", "", conv)
    conv = re.sub("\/\/\/\/", "", conv)
    conv = re.sub("\.\.\.\.", "", conv)
    conv = re.sub("\*\*\*\*", "", conv)
    conv = re.sub("####", "", conv)
    conv = re.sub("\?\?\?\?", "", conv)
    conv = re.sub("@[a-zA-Z_0-9\S]+", "", conv)
    
    if random() < 0.98:
        conv = re.sub("thank you for creating this issue. we will troubleshoot it as soon as we can. Info for maintainers".lower(), "", conv)
    if random() < 0.98:
        conv = re.sub("thank you for creating this issue. i am the selenium assistant bot , i triage issues in this repository. if i can't do it, i label it to help maintainers identify issues that need triaging. i am an open source project , post bugs or ideas here!", "", conv)

    suber_350 = "If you are asking a question, a better way to address this is:  Send questions and support requests to the Selenium user group  Post them to StackOverflow  Join us in the IRC/Slack channel where the community can help you as well If you think this is incorrect, please feel free to open a new issue."
    if suber_350.lower() in conv:
        if random() < 0.98:
            conv = re.sub(suber_350.lower(), "", conv)
    suber_500 = "thank you for your contribution"
    if suber_500.lower() in conv:
        if random() < 0.98:
            conv = re.sub(suber_500.lower(), "", conv)
    suber_386 = "It seems this issue is not using any of the supported templates  Supported issue types are (they start with):  (bugs found in a recent release)  (a useful feature you would like to propose)  (a supported feature is not working anymore) Issue templates help this project to stay in shape, please use them and fill them out completely. By doing that you are helping the project because the community and maintainers can provide prompt feedback, and potentially solve the issue"
    if suber_386.lower() in conv:
        suber_386 = "It seems this issue is not using any of the supported templates  Supported issue types are \(they start with\):  \(bugs found in a recent release\)  \(a useful feature you would like to propose\)  \(a supported feature is not working anymore\) Issue templates help this project to stay in shape, please use them and fill them out completely. By doing that you are helping the project because the community and maintainers can provide prompt feedback, and potentially solve the issue"
        if random() < 0.98:
            conv = re.sub(suber_386.lower(), "", conv)
    suber_600 = "This issue has been automatically locked since there has not been any recent activity after it was closed. Please open a new issue for related bugs."
    if suber_600.lower() in conv:
        if random() < 0.98:
            conv = re.sub(suber_600.lower(), "", conv)
    
    # For code snippets, concatenate them to 1 word ??

    print('conv.split("_eot_") is', conv.split("_eot_"))
    convs.append(conv.split("_eot_"))
    count += 1
# print(convs)

# Write data to Csv

In [18]:
fields = ['Response', 'Context/0', 'Context/1', 'Context/2', 'Context/3'] 

# data rows of csv file 
rows = []
for con in convs:
    turns = [""]*5
    for i in range(5):
        try:
            if con[i]:
                turns[i] = con[i]
        except:
            continue
    rows.append(list(reversed(turns)))
print(len(rows))
# print(rows)
with open('selenium-full-cleaned-5-turns.csv', 'w', encoding="utf-8") as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

4003


# Preprocess data (remove html tags) - work from Crystal

In [ ]:
# import html

# def clean_html(text, strip_numeric=False):
#     """
#     Decode, remove characters. Words and numbers that are not html tags syntax.
#     INPUT
#     text            string of text
#     strip_numeric   bool, indicates if numeric type should be stripped
#     OUTPUT
#     cleaned, stripped text as string
#     """
#     cleaned = html.unescape(
#         text
#     )  # Convert all named and numeric character references (e.g. &gt;, &#62;, &x3e;) in the string s to the corresponding unicode characters
#     # First we remove inline JavaScript/CSS:
#     # cleaned = cleaned.lower()
#     cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", cleaned.strip())
#     # Then we remove html comments. This has to be done before removing regular
#     # tags since comments can contain '>' characters.
#     cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
#     # Next we can remove the remaining tags:
#     # Next we can remove the remaining tags with the exception of meta content for description and keywords:
#     cleaned = re.sub(r"(?s)<(?!meta content=\"[^<>]+name=\"(?:keywords|description)\").*?>", " ", cleaned)
#     # Now we want to clean the data to take out the meta content tags
#     cleaned = re.sub(r"(?:(<meta content.?\=\")|(name=\"(?:description|keywords)\"\/>))", "", cleaned)
#     # In case of any other punctuation leftover, we can remove them:
#     # cleaned = re.sub(r"[^\w]", " ", cleaned)

#     # Restrict characters
#     if strip_numeric:
#         cleaned = re.sub(r'[^a-zA-Z!,\(\)\.]', ' ', cleaned)
#     else:
#         # cleaned = re.sub(r'[^a-zA-Z0-9!,\(\)\.]', ' ', cleaned)
#         pass

#     cleaned = replace_two_or_more(cleaned)  # replace repetitions with a single instance
#     # Finally, we deal with whitespace
#     cleaned = re.sub(r"&nbsp;", " ", cleaned)
#     # cleaned = re.sub(r"  ", " ", cleaned)
#     cleaned = re.sub(r'[\s]+', ' ', cleaned)
#     cleaned = ' '.join(cleaned.split())  # remove whitespace for readability
#     return cleaned.strip()